In [ ]:
from datetime import datetime
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
import os
import re
from docx2pdf import convert
import win32com.client as win32
from win32com.client import constants

This section imports necessary libraries for document creation, PDF conversion, and Google Drive interaction.

In [ ]:

def sanitize_filename(filename):
    """
    Sanitizes the filename to remove or replace invalid characters.
    """
    return re.sub(r'[^a-zA-Z0-9_\-]', '_', filename)

##### Function: `sanitize_filename`

Sanitizes a given filename to remove or replace invalid characters with underscores (`_`).

##### Parameters:
- `filename`: A string representing the filename to be sanitized.

##### Returns:
- A sanitized version of the filename where any characters not matching `[a-zA-Z0-9_\-]` are replaced with underscores (`_`).




In [ ]:
def set_font(run, font_name, font_size, bold=False):
    run.font.name = font_name
    run.font.size = Pt(font_size)
    run.bold = bold
    r = run._element
    r.rPr.rFonts.set(qn('w:eastAsia'), font_name)

Functionality:

Sets the font.name and font.size properties of the run to the specified font_name and font_size, respectively.
If bold is True, it sets run.bold to True, making the font bold.
Ensures consistent font settings across the run, including support for East Asian fonts (w:eastAsia).
This function is useful for applying consistent font styles to text runs within a Microsoft Word document, allowing flexibility with bold formatting when needed.

In [ ]:
def create_cover_letter(position, company_name, company_address, sector, achievements, strengths, font_name='Arial', font_size=12):
    " fuction to create the desired template, with the insertion of customized areas "
    # Create a new Document
    doc = Document()

    # Add the Date
    date_paragraph = doc.add_paragraph()
    current_date = datetime.now().strftime("%B %d, %Y")
    run = date_paragraph.add_run(current_date)
    set_font(run, font_name, font_size)

    # Add a blank line
    doc.add_paragraph('\n')
    
    # Example content
    content = [
    f"I am writing to apply for the position of {position} at {company_name}{position} at {company_name}. I am particularly drawn to {company_name} innovative approach in the {sector} field."
        f"{company_name} has notably achieved {achievements}This is particularly impressive and resonates with my ambition to contribute to such a dynamic environment with my analytical skills.",
    "To demonstrate my proactive approach and technical skills, I have automated the creation of reports using Python scripts. This allows me to efficiently generate tailored reports consistently. Recognizing the importance of efficiency and precision in data analysis, I developed Python scripts to automate various data processes.",
    "This script leverages the pandas library to analyze data dynamically, ensuring that each report is meticulously tailored to specific requirements. The scripts I developed include several key features:"
     ]

    # Loop through the content paragraphs
    for paragraph_text in content:
    p = doc.add_paragraph()

    # Check if the specific text is present in the paragraph
    if "I have automated the creation of reports using Python scripts" in paragraph_text:
        parts = paragraph_text.split("I have automated the creation of reports using Python scripts")
        if parts[0]:
            run = p.add_run(parts[0])
            set_font(run, "Arial", 12)  # Specify your desired font and size
        run = p.add_run("I have automated the creation of reports using Python scripts")
        set_font(run, "Arial", 12, bold=True)  # Make this specific text bold
        if parts[1]:
            run = p.add_run(parts[1])
            set_font(run, "Arial", 12)  # Specify your desired font and size
    else:
        run = p.add_run(paragraph_text)
        set_font(run, "Arial", 12)  # Specify your desired font and size

 # Add bullet points
    bullet_points = [
        "Dynamic insertion of company name, notable achievements, and relevant strengths.",
        "Customizable font and formatting options.",
        "Automated date insertion and address formatting.",
        "`python-docx` library for document creation and manipulation."
    ]

    for point in bullet_points:
        p = doc.add_paragraph()
        p.style = 'List Bullet'
        run = p.add_run(point)
        set_font(run, font_name, font_size)

    # Add GitHub repository link
    github_paragraph = doc.add_paragraph()
    github_text = (
        "You can view the full script on my [GitHub repository](https://github.com/name/repo)."
    )
    run = github_paragraph.add_run(github_text)
    run.font.bold = True
    run.font.underline = True
    set_font(run, font_name, font_size)

    # Add closing statement
    closing_paragraph = doc.add_paragraph()
    closing_text = (
        f"I am confident that my technical skills, combined with {strengths}, make me a strong candidate for this position. "
        f"I am eager to contribute to {company_name} and I am available for an interview at your earliest convenience. "
        "Thank you for considering my application."
    )
    run = closing_paragraph.add_run(closing_text)
    set_font(run, font_name, font_size)

    # Add signature
    signature_paragraph = doc.add_paragraph()
    run = signature_paragraph.add_run('Sincerely,')
    set_font(run, font_name, font_size)



##### The snippet above creates the template of the letter

In [ ]:
    # Sanitize the filename
    sanitized_company_name = sanitize_filename(company_name)
    doc_filename = f'cover_letter_{sanitized_company_name}.docx'

    # Save the document
    try:
        doc.save(doc_filename)
    except PermissionError:
        print(f"Error: Permission denied while saving {doc_filename}. Please close the file if it's open and try again.")
        return

    # Convert to PDF
    pdf_file_name = f"cover_letter_stefania_chiesa_{sanitized_company_name}.pdf"
    try:
        convert(doc_filename, pdf_file_name)
    except Exception as e:
        print(f"Error converting to PDF: {e}")
        return


##### The snippet above demonstrates the process of sanitizing a filename, saving a document, and converting it to PDF format in Python, handling potential errors along the way.

In [ ]:
  # Get the full path to the saved PDF document
    full_path = os.path.abspath(pdf_file_name)
    print(f"Cover letter saved as {full_path}")

    # Upload to Google Drive
    upload_to_google_drive(full_path, pdf_file_name)

###### Get the full path to the saved PDF document

The `os.path.abspath` function is used to obtain the absolute path of the saved PDF file (`pdf_file_name`). This ensures that the path returned is complete and includes the drive letter (on Windows) or root directory (on Unix-like systems) along with all intermediate directories. For example, if `pdf_file_name` is "cover_letter_stefania_chiesa_XYZ_Company__Inc_.pdf", `full_path` will hold the complete path to this file on the file system.




In [ ]:
def upload_to_google_drive(file_path, file_name):
    SCOPES = ['https://www.googleapis.com/auth/drive.file']
    credentials = service_account.Credentials.from_service_account_file('XXXX.json', scopes=SCOPES)
    service = build('drive', 'v3', credentials=credentials)

    file_metadata = {'name': file_name}
    media = MediaFileUpload(file_path, mimetype='application/pdf')

    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print(f'File ID: {file.get("id")}')
    share_file(file.get("id"))

###### Upload to Google Drive

The `upload_to_google_drive` function facilitates uploading a file to Google Drive using the Google Drive API. Below is a breakdown of its components:

SCOPES: Defines the access scope required for the Google Drive API. In this case, drive.file scope is used, which allows the application to create, open, and edit files on the user's Drive.

credentials: Loads the service account credentials (vvv.json) to authenticate and authorize access to the Google Drive API with the specified scopes.

service: Initializes the Google Drive service using the build function from the googleapiclient.discovery module. This service object is used to interact with Google Drive API methods.

file_metadata: Specifies the metadata for the file being uploaded. Here, the name key is set to file_name, which represents the name of the file as it will appear in Google Drive.

media: Creates a MediaFileUpload object initialized with file_path and mimetype='application/pdf'. This object represents the file content to be uploaded along with its MIME type.


In [ ]:
def share_file(file_id):
    SCOPES = ['https://www.googleapis.com/auth/drive']
    credentials = service_account.Credentials.from_service_account_file('XXXX.json', scopes=SCOPES)
    service = build('drive', 'v3', credentials=credentials)

    try:
        # Define new permissions
        new_permission = {
            'type': 'anyone',  # This makes the file accessible by anyone with the link
            'role': 'reader',  # This gives read-only access
        }
        service.permissions().create(
            fileId=file_id,
            body=new_permission
        ).execute()
        print(f"File with ID {file_id} is now shared publicly.")
        
        # Retrieve the web view link
        file = service.files().get(fileId=file_id, fields='webViewLink').execute()
        print(f"File Web View Link: {file.get('webViewLink')}")
    except Exception as e:
        print(f"Error sharing file: {e}")


Permissions Creation: Creates a new permission for the file specified by file_id. The new_permission dictionary specifies that anyone with the link (type='anyone') has read-only access (role='reader').

service.permissions().create: Calls the create method of the Google Drive service to set the permissions defined in new_permission for the file identified by file_id.

fileId=file_id: Specifies the ID of the file for which permissions are being set.

fields='webViewLink': Specifies that the API response should include the webViewLink, which is the link to view the file in a web browser.

Execute and Print: Executes the API request to set permissions and prints a confirmation message that the file is now shared publicly, along with the web view link.

Exception Handling: Catches any exceptions that may occur during the process, such as errors related to API requests, and prints an error message indicating the issue.

##### This function provides a streamlined approach to sharing files on Google Drive programmatically, ensuring that files can be easily accessed and viewed by others with the appropriate permissions.

In [ ]:
# Example usage
create_cover_letter("Position Title"," Max Power Company", " picasso street 12, Haifa", "industrial", "Significant milestone of becoming publicly traded last year", " great ability to work ", font_name="Arial", font_size=10)

###### Example of usage